In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)

device =  cpu


In [ ]:
pip install --upgrade pip

In [ ]:
!git clone https://github.com/gregdurrett/fp-dataset-artifacts.git

fatal: destination path 'fp-dataset-artifacts' already exists and is not an empty directory.


In [ ]:
pip install -r ./fp-dataset-artifacts/requirements.txt

In [ ]:
pip install -U kaleido

In [ ]:
cd fp-dataset-artifacts/

/content/fp-dataset-artifacts


In [ ]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
from datasets import load_dataset
datasets_squad = load_dataset("squad")

# Save the 'train' split to JSON
datasets_squad["train"].to_json('squad_train.json')

# Save the 'train' split to JSON
datasets_squad["validation"].to_json('squad_validation.json')

Creating json from Arrow format:   0%|          | 0/88 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

11139706

In [ ]:
pip install pandas matplotlib plotly nltk

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import json
nltk.download('punkt')

# Load the JSON dataset
def load_json_dataset(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            try:
                parsed_line = json.loads(line)
                data.append(parsed_line)
            except json.JSONDecodeError as e:
                print(f"Error parsing line: {e}")
                # You can add more sophisticated error handling here if needed
    return pd.DataFrame.from_records(data)

# Clean the dataset
def clean_text(text):
    # Implement any specific text cleaning here
    return text

# Histogram of word counts in questions
def plot_word_count_histogram(questions):
    word_counts = questions.apply(lambda x: len(x.split()))
    plt.hist(word_counts, bins=20)
    plt.title('Histogram of Question Word Counts')
    plt.xlabel('Number of Words')
    plt.ylabel('Frequency')
    plt.show()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from datasets import load_dataset
from nltk.util import ngrams
from nltk import word_tokenize
import nltk
import plotly.express as px
import plotly.io as pio
nltk.download('punkt')

def get_max_ngram_size(question, context):
    question_tokens = word_tokenize(question)
    context_tokens = word_tokenize(context)
    max_n = min(len(question_tokens), len(context_tokens))

    for n in range(max_n, 0, -1):
        question_ngrams = set(ngrams(question_tokens, n))
        context_ngrams = set(ngrams(context_tokens, n))
        if question_ngrams.intersection(context_ngrams):
            return n
    return 0

def plot_ngram_distribution(dataset, file_path, bar_color):
    ngram_sizes = []

    for example in dataset:
        max_ngram = get_max_ngram_size(example['question'], example['context'])
        ngram_sizes.append(max_ngram)

    ngram_counter = Counter(ngram_sizes)
    total = sum(ngram_counter.values())
    normalized_ngrams = {ngram: count / total for ngram, count in ngram_counter.items()}

    # Calculate mean and standard deviation
    mean_ngram = np.mean(ngram_sizes)
    std_ngram = np.std(ngram_sizes)

    fig = px.bar(x=list(normalized_ngrams.keys()), y=list(normalized_ngrams.values()),
                 labels={'x': 'N-gram Size', 'y': 'Normalized Frequency'},
                 title='Normalized N-gram Size Distribution in Questions Against Context',
                 color_discrete_sequence=[bar_color] if bar_color else None)

    # Remove gridlines and update styles
    fig.update_layout(xaxis=dict(showgrid=False, title_font=dict(size=24, color='black', family="Arial, bold"),
                                 tickfont=dict(size=18, color='black', family="Arial")),
                      yaxis=dict(showgrid=False, title_font=dict(size=24, color='black', family="Arial, bold"),
                                 tickfont=dict(size=18, color='black', family="Arial")),
                      font=dict(size=14, color="black", family="Arial, bold"),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)')

    # Set x-axis limit
    fig.update_xaxes(range=[0, 10])

    # Add text for mean and standard deviation
    fig.add_annotation(text=f"μ={mean_ngram:.2f}, σ={std_ngram:.2f}",
                       xref="paper", yref="paper",
                       x=0.95, y=0.95, showarrow=False, font=dict(size=24, color="black", family="Arial, bold"))

    pio.write_image(fig, file_path)
    fig.show()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Plotting the N-gram of the questions vs the contexts for SQuAD trian dataset.

In [ ]:
# Load the SQuAD dataset
import numpy as np
datasets_squad = load_dataset("squad")
# plot_ngram_distribution(datasets_squad['train'], "SQuAD_Train_N_Gram.png", "dodgerblue")

Plotting the N-gram of the questions vs the contexts for SQuAD validation dataset.

In [ ]:
plot_ngram_distribution(datasets_squad['validation'], "SQuAD_validation_N_Gram.png", "dodgerblue")

In [ ]:
from datasets import load_dataset
import pandas as pd
import plotly.express as px
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import plotly.io as pio

# Combine train and validation questions
train_questions = datasets_squad['train']['question']
validation_questions = datasets_squad['validation']['question']

def process_questions(questions):
    processed_data = []
    for question in questions:
        words = word_tokenize(question)[:3]
        for i in range(2, len(words)):
            grandparent = words[i-2]
            parent = words[i-1]
            word = words[i]
            processed_data.append({'grandparent': grandparent, 'parent': parent, 'word': word})
    return processed_data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import plotly.io as pio
# Sort by frequency and take the top 100 rows

processed_data_SQuAD = process_questions(train_questions)

# Creating a DataFrame for the sunburst plot
SQuAD_training_DF = pd.DataFrame(processed_data_SQuAD)

# Aggregate and count the frequency of each triplet
SQuAD_training_agg_DF = SQuAD_training_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')
SQuAD_training_agg_DF_head = SQuAD_training_agg_DF.sort_values('frequency', ascending=False).head(100)


In [ ]:

# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    SQuAD_training_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="Sunburst Plot of Word Triplets in Questions"
)
fig.update_layout(width=800, height=800)

# Save the figure as a PNG file
file_path = "SQuAD_Train_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()

In [ ]:
import plotly.io as pio
# Sort by frequency and take the top 100 rows

processed_data_SQuAD_Validation = process_questions(train_questions)

# Creating a DataFrame for the sunburst plot
SQuAD_validation_DF = pd.DataFrame(processed_data_SQuAD_Validation)

# Aggregate and count the frequency of each triplet
SQuAD_validation_agg_DF = SQuAD_validation_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')
SQuAD_validation_agg_DF_head = SQuAD_validation_agg_DF.sort_values('frequency', ascending=False).head(100)


In [ ]:

# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    SQuAD_validation_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="Sunburst Plot of Word Triplets in Questions"
)
fig.update_layout(width=800, height=800)

# Save the figure as a PNG file
file_path = "SQuAD_Validation_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()

In [ ]:
import plotly.express as px

def plot_grandparent_frequency_bar_graph(df):
    # Aggregate the frequencies by grandparent
    grandparent_freq = df.groupby('grandparent')['frequency'].sum().reset_index()
    grandparent_freq = grandparent_freq.sort_values('frequency', ascending=False)
    # Create the bar graph
    fig = px.bar(grandparent_freq.head(10), x='grandparent', y='frequency', title='Frequency of Question Words')
    fig.show()

In [ ]:
!unzip aqa_v1.zip

Archive:  aqa_v1.zip
   creating: aqa_v1-2/
replace __MACOSX/._aqa_v1-2? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls

In [ ]:
dataset_aqa_combined_train = load_dataset ("json", data_files="./aqa_v1/combined/train.json")
dataset_aqa_combined_test = load_dataset ("json", data_files="./aqa_v1/combined/test.json")

In [ ]:
import json
from datasets import Dataset

def convert_to_dataset_dict(file_path, output_file_path, split):
    # Read the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Extract and reformat the data
    formatted_data = []  # Initialize as a list
    for article in data["data"]:
        title = article["title"]
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                id = qa["id"]
                question = qa["question"]
                # Process answers
                answers = {"text": [], "answer_start": []}
                if "answers" in qa and qa["answers"]:
                    for answer in qa['answers']:
                        answers["text"].append(answer["text"])
                        answers["answer_start"].append(answer["answer_start"])
                # Append the extracted data in the required format
                formatted_item = {
                    "id": id,
                    "title": title,
                    "context": context,
                    "question": question,
                    "answers": answers
                }
                formatted_data.append(formatted_item)

    # Write the formatted data to a new JSON file with each entry on a new line
    with open(output_file_path, 'w') as outfile:
        for item in formatted_data:
            json.dump(item, outfile)
            outfile.write('\n')

    # Convert the formatted data into a Dataset object
    dataset = Dataset.from_dict({k: [dic[k] for dic in formatted_data] for k in formatted_data[0]})

    return dataset

In [ ]:
datasets_train_aqa = convert_to_dataset_dict("/content/fp-dataset-artifacts/aqa_v1/combined/train.json", "./train_aqa.json", "train")
datasets_train_aqa

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 30000
})

In [ ]:
datasets_validation_aqa = convert_to_dataset_dict("/content/fp-dataset-artifacts/aqa_v1/combined/dev.json", "./dev_aqa.json", "validation")
datasets_validation_aqa

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 3000
})

In [ ]:
from datasets import DatasetDict
dataset_aqa_combined = DatasetDict({
    'train': datasets_train_aqa,
    'validation': datasets_validation_aqa
})
dataset_aqa_combined

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 3000
    })
})

In [ ]:
# Combine train and validation questions
aqa_train_questions = dataset_aqa_combined['train']['question']
aqa_validation_questions = dataset_aqa_combined['validation']['question']

processed_data_aqa_train = process_questions(aqa_train_questions)

# Creating a DataFrame for the sunburst plot
AQA_Train_DF = pd.DataFrame(processed_data_aqa_train)

# Convert 'grandparent', 'parent', and 'word' columns to lowercase to avoid case-sensitive distinctions
AQA_Train_DF['grandparent'] = AQA_Train_DF['grandparent'].str.lower()
AQA_Train_DF['parent'] = AQA_Train_DF['parent'].str.lower()
AQA_Train_DF['word'] = AQA_Train_DF['word'].str.lower()

# Aggregate and count the frequency of each triplet
AQA_Train_agg_DF = AQA_Train_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')

In [ ]:
import plotly.io as pio

# Sort by frequency and take the top 100 rows
AQA_Train_agg_DF_head = AQA_Train_agg_DF.sort_values('frequency', ascending=False)

# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    AQA_Train_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="Sunburst Plot of Word Triplets in Questions"
)
fig.update_layout(width=800, height=800)

# Save the figure as a PNG file
file_path = "AQA_Train_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()


In [ ]:
# validation questions
aqa_validation_questions = dataset_aqa_combined['validation']['question']

processed_data_aqa_validation = process_questions(aqa_validation_questions)

# Creating a DataFrame for the sunburst plot
AQA_Validation_DF = pd.DataFrame(processed_data_aqa_validation)

# Convert 'grandparent', 'parent', and 'word' columns to lowercase to avoid case-sensitive distinctions
AQA_Validation_DF['grandparent'] = AQA_Validation_DF['grandparent'].str.lower()
AQA_Validation_DF['parent'] = AQA_Validation_DF['parent'].str.lower()
AQA_Validation_DF['word'] = AQA_Validation_DF['word'].str.lower()


# Aggregate and count the frequency of each triplet
AQA_Validation_agg_DF = AQA_Validation_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')

In [ ]:
import plotly.io as pio
# Sort by frequency and take the top 100 rows
AQA_Validation_agg_DF_head = AQA_Validation_agg_DF.sort_values('frequency', ascending=False)

# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    AQA_Validation_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="Sunburst Plot of Word Triplets in Questions"
)
fig.update_layout(width=800, height=800)

# Save the figure as a PNG file
file_path = "AQA_Validation_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()

In [ ]:
plot_ngram_distribution(dataset_aqa_combined['train'], "AQA_Train_N_Gram.png", "firebrick")

In [ ]:
plot_ngram_distribution(dataset_aqa_combined['validation'], "AQA_Validation_N_Gram.png", "firebrick")

In [ ]:
datasets_squad_adversarial = load_dataset("squad_adversarial", "AddSent")

# validation questions
squad_adversarial_validation_questions = datasets_squad_adversarial['validation']['question']

processed_data_squad_adversarial_validation = process_questions(squad_adversarial_validation_questions)

# Creating a DataFrame for the sunburst plot
SQuAD_Adversarial_Validation_DF = pd.DataFrame(processed_data_squad_adversarial_validation)

# Convert 'grandparent', 'parent', and 'word' columns to lowercase to avoid case-sensitive distinctions
SQuAD_Adversarial_Validation_DF['grandparent'] = SQuAD_Adversarial_Validation_DF['grandparent'].str.lower()
SQuAD_Adversarial_Validation_DF['parent'] = SQuAD_Adversarial_Validation_DF['parent'].str.lower()
SQuAD_Adversarial_Validation_DF['word'] = SQuAD_Adversarial_Validation_DF['word'].str.lower()


# Aggregate and count the frequency of each triplet
SQuAD_Adversarial_Validation_agg_DF = SQuAD_Adversarial_Validation_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')

In [ ]:
import plotly.io as pio
# Sort by frequency and take the top 100 rows
SQuAD_Adversarial_Validation_agg_DF_head = SQuAD_Adversarial_Validation_agg_DF.sort_values('frequency', ascending=False)



In [ ]:
# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    SQuAD_Adversarial_Validation_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="Sunburst Plot of Word Triplets in Questions"
)
fig.update_layout(width=800, height=800)

# Save the figure as a PNG file
file_path = "SQuAD_Adversarial_Validation_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()

In [ ]:
plot_ngram_distribution(datasets_squad_adversarial['validation'], "SQuAD_Adversarial_Validation_N_Gram.png", "darkgreen")

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset_AdQuest_parent = load_dataset("mbartolo/synQA")

# Define a filter function
def starts_with_specific_words(example):
    question_start_words = ["when", "where", "which", "how many"]
    question_lower = example['question'].lower()
    return any(question_lower.startswith(word) for word in question_start_words)

# Filter the dataset
AdQuest = dataset_AdQuest_parent['train'].filter(starts_with_specific_words)

# Save the filtered 'train' split to JSON
AdQuest.to_json('AdQuest_train.json')

# Combine train and validation questions
AdQuest_train_questions = AdQuest['question']

processed_data_AdQuest_train = process_questions(AdQuest_train_questions)

# Creating a DataFrame for the sunburst plot
AdQuest_Train_DF = pd.DataFrame(processed_data_AdQuest_train)

# Convert 'grandparent', 'parent', and 'word' columns to lowercase to avoid case-sensitive distinctions
AdQuest_Train_DF['grandparent'] = AdQuest_Train_DF['grandparent'].str.lower()
AdQuest_Train_DF['parent'] = AdQuest_Train_DF['parent'].str.lower()
AdQuest_Train_DF['word'] = AdQuest_Train_DF['word'].str.lower()

# Aggregate and count the frequency of each triplet
AdQuest_Train_agg_DF = AdQuest_Train_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')

Generating train split:   0%|          | 0/314811 [00:00<?, ? examples/s]

Filter:   0%|          | 0/314811 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/68 [00:00<?, ?ba/s]

In [ ]:
import plotly.io as pio

# Sort by frequency and take the top 100 rows
AdQuest_Train_agg_DF_head = AdQuest_Train_agg_DF.sort_values('frequency', ascending=False).head(10000)

# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    AdQuest_Train_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="Sunburst Plot of Word Triplets in Questions"
)
fig.update_layout(width=800, height=800,
                  paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

# Save the figure as a PNG file
file_path = "AdQuest_Train_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()

In [ ]:
# Load the AdQuest dataset
plot_ngram_distribution(AdQuest, "AdQuest_Train_N_Gram.png", "goldenrod")

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.io as pio

def plot_combined_grandparent_percentage_bar_graph(dfs, dataset_names, file_path_words):
    if len(dfs) != len(dataset_names):
        raise ValueError("The number of datasets and dataset names must be the same")

    combined_df = pd.DataFrame()

    for df, name in zip(dfs, dataset_names):
        # Convert 'grandparent' to lowercase for case-insensitive aggregation
        df['grandparent'] = df['grandparent'].str.lower()

        # Aggregate frequencies by grandparent
        grandparent_freq = df.groupby('grandparent')['frequency'].sum().reset_index()
        total_freq = grandparent_freq['frequency'].sum()  # Total frequency in this dataset
        grandparent_freq['percentage'] = (grandparent_freq['frequency'] / total_freq) * 100
        grandparent_freq = grandparent_freq.sort_values('percentage', ascending=False).head(10)
        grandparent_freq['dataset'] = name  # Add dataset name as a column
        combined_df = pd.concat([combined_df, grandparent_freq])

    # Create the bar graph
    fig = px.bar(combined_df, x='grandparent', y='percentage', color='dataset',
                 barmode='group', title='Top 10 Grandparent Word Percentages Across Datasets')

    pio.write_image(fig, file_path_words)
    fig.show()

plot_combined_grandparent_percentage_bar_graph(
    [SQuAD_training_agg_DF_head, AQA_Train_agg_DF, AdQuest_Train_agg_DF],
    ["SQuAD", "AQA", "AdQuest"], "Question_Words_Datasets.png"
)

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.io as pio

def plot_combined_grandparent_percentage_bar_graph(dfs, dataset_names, file_path_words):
    if len(dfs) != len(dataset_names):
        raise ValueError("The number of datasets and dataset names must be the same")

    combined_df = pd.DataFrame()
    # List of question words to include in the specified order
    question_words = ["what", "when", "how", "in", "who", "where", "why", "which", "other"]

    for df, name in zip(dfs, dataset_names):
        # Convert 'grandparent' to lowercase for case-insensitive aggregation
        df['grandparent'] = df['grandparent'].str.lower()

        # Categorize all other words as 'other'
        df['grandparent'] = df['grandparent'].apply(lambda x: x if x in question_words[:-1] else 'other')

        # Aggregate frequencies by grandparent
        grandparent_freq = df.groupby('grandparent')['frequency'].sum().reset_index()
        total_freq = grandparent_freq['frequency'].sum()  # Total frequency in this dataset
        grandparent_freq['percentage'] = (grandparent_freq['frequency'] / total_freq) * 100
        grandparent_freq['dataset'] = name  # Add dataset name as a column
        combined_df = pd.concat([combined_df, grandparent_freq])

    # Define color map for datasets
    color_map = {
        "SQuAD": "dodgerblue",
        "AQA": "firebrick",
        "AdversarialQA": "darkgreen",
        "AdQuest": "goldenrod"
    }

    # Set the order of categories for the x-axis
    fig = px.bar(combined_df, x='grandparent', y='percentage', color='dataset',
                 barmode='group', title='Grandparent Word Percentages Across Datasets',
                 category_orders={"grandparent": question_words},
                 color_discrete_map=color_map)

    # Update layout for larger, bold labels and ticks
    fig.update_layout(xaxis=dict(showgrid=False, title_font=dict(size=48, color='black', family="Arial, bold"),
                                 tickfont=dict(size=36, color='black', family="Arial")),
                      yaxis=dict(showgrid=False, title_font=dict(size=48, color='black', family="Arial, bold"),
                                 tickfont=dict(size=36, color='black', family="Arial")),
                      font=dict(size=28, color="black", family="Arial, bold"),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)')
    pio.write_image(fig, file_path_words, width=1920, height=1080)
    fig.show()

# Example usage (replace with your actual DataFrames)
plot_combined_grandparent_percentage_bar_graph(
    [SQuAD_training_agg_DF_head, AQA_Train_agg_DF],
    ["SQuAD", "AQA"], "Question_Words_Datasets_2.png"
)


In [ ]:
import plotly.graph_objects as go

# Data
question_types = ["Explicit", "Paraphrasing", "Multi-hop", "Negation", "Temporal", "Spatial", "Implicit"]
SQuAD_Qs = [57, 16.4, 2.4, 0, 0.5, 0, 10]
AQA_Qs = [5.7, 27, 18, 1.8, 8, 3, 13.4]

# Creating two bar graphs
fig = go.Figure()

# Adding SQuAD data
fig.add_trace(go.Bar(
    x=question_types,
    y=SQuAD_Qs,
    name='SQuAD',
    marker_color='dodgerblue'
))

# Adding AQA data
fig.add_trace(go.Bar(
    x=question_types,
    y=AQA_Qs,
    name='AQA',
    marker_color='firebrick'
))

# Updating layout
fig.update_layout(
    title='Comparison of Question Types in SQuAD and AQA Datasets',
    xaxis=dict(
        showgrid=False,
        title='Question Types',
        titlefont=dict(size=48, color='black', family="Arial, bold"),
        tickfont=dict(size=36, color='black', family="Arial, bold"),
        tickmode='array',
        tickvals=question_types,
        ticktext=question_types,
        tickangle=0
    ),
    yaxis=dict(
        showgrid=False,
        title='Percentage (%)',
        titlefont=dict(size=48, color='black', family="Arial, bold"),
        tickfont=dict(size=36, color='black', family="Arial, bold")
    ),
    legend=dict(
        x=0.8,
        y=1.0,
        bgcolor='rgba(0, 0, 0, 0)',
        bordercolor='rgba(0, 0, 0, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    plot_bgcolor='rgba(0,0,0,0)', # transparent background
)

pio.write_image(fig, "question_type.png",  width=1920, height=1080)
fig.show()


In [ ]:
!mkdir Data_Analysis
!cp *.png ./Data_Analysis/
!cp *.json ./Data_Analysis/

mkdir: cannot create directory ‘Data_Analysis’: File exists


In [ ]:
!zip -r Data_Analysis_for_Dataset_Comparison_2.zip ./Data_Analysis

updating: Data_Analysis/ (stored 0%)
updating: Data_Analysis/SQuAD_validation_N_Gram.png (deflated 13%)
updating: Data_Analysis/SQuAD_Adversarial_Validation_Word_Distribution.png (deflated 1%)
updating: Data_Analysis/SQuAD_Validation_Word_Distribution.png (deflated 2%)
updating: Data_Analysis/AdQuest_Train_Word_Distribution.png (deflated 1%)
updating: Data_Analysis/AQA_Train_N_Gram.png (deflated 15%)
updating: Data_Analysis/dev_aqa.json (deflated 88%)
updating: Data_Analysis/squad_validation.json (deflated 89%)
updating: Data_Analysis/SQuAD_Train_N_Gram.png (deflated 13%)
updating: Data_Analysis/SQuAD_Train_Word_Distribution.png (deflated 2%)
updating: Data_Analysis/AQA_Validation_N_Gram.png (deflated 15%)
updating: Data_Analysis/AdQuest_Train_N_Gram.png (deflated 14%)
updating: Data_Analysis/train_aqa.json (deflated 90%)
updating: Data_Analysis/AQA_Train_Word_Distribution.png (deflated 1%)
updating: Data_Analysis/AQA_Validation_Word_Distribution.png (deflated 1%)
updating: Data_Analys

In [ ]:
!cp Data_Analysis_for_Dataset_Comparison_2.zip /content/drive/MyDrive/NLP/

In [ ]:
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
import plotly.express as px
import plotly.io as pio
import numpy as np
from collections import Counter

nltk.download('punkt')

def get_length_distribution(data, key):
    if key == "question":
        lengths = [len(word_tokenize(item[key])) for item in data]
    else:
        lengths = [len(word_tokenize(item[key]['text'][0])) for item in data]
    return lengths

def plot_histogram(data, title, x_label, file_path, bar_color, x_axis_limit):
    counter = Counter(data)
    total = sum(counter.values())
    normalized_data = {length: count / total for length, count in counter.items()}

    # Calculate mean and standard deviation
    mean_length = np.mean(list(data))
    std_length = np.std(list(data))

    fig = px.bar(x=list(normalized_data.keys()), y=list(normalized_data.values()),
                 labels={'x': x_label, 'y': 'Normalized Frequency'},
                 title=title, color_discrete_sequence=[bar_color] if bar_color else None)

    # Remove gridlines and update styles
    fig.update_layout(xaxis=dict(showgrid=False, title_font=dict(size=24, color='black', family="Arial, bold"),
                                 tickfont=dict(size=18, color='black', family="Arial")),
                      yaxis=dict(showgrid=False, title_font=dict(size=24, color='black', family="Arial, bold"),
                                 tickfont=dict(size=18, color='black', family="Arial")),
                      font=dict(size=14, color="black", family="Arial, bold"),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)')

    fig.update_xaxes(range=[0, x_axis_limit])

    # Add text for mean and standard deviation
    fig.add_annotation(text=f"μ={mean_length:.2f}, σ={std_length:.2f}",
                       xref="paper", yref="paper",
                       x=0.95, y=0.95, showarrow=False, font=dict(size=24, color="black", family="Arial, bold"))

    pio.write_image(fig, file_path)
    fig.show()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Plot question length distribution
question_lengths = get_length_distribution(datasets_squad['train'], 'question')
plot_histogram(question_lengths, 'Question Length Distribution', 'Question Length', 'SQuAD_Train_Question_Lengths.png', 'dodgerblue', 30)

# Plot answer length distribution
answer_lengths = get_length_distribution(datasets_squad['train'], 'answers')  # Assuming 'answer' is the key for answers
plot_histogram(answer_lengths, 'Answer Length Distribution', 'Answer Length', 'SQuAD_Train_Answer_Lengths.png', 'dodgerblue', 15)


In [ ]:
# Plot question length distribution
question_lengths = get_length_distribution(datasets_squad['validation'], 'question')
plot_histogram(question_lengths, 'Question Length Distribution', 'Question Length', 'SQuAD_Validation_Question_Lengths.png', 'dodgerblue', 30)

# Plot answer length distribution
answer_lengths = get_length_distribution(datasets_squad['validation'], 'answers')  # Assuming 'answer' is the key for answers
plot_histogram(answer_lengths, 'Answer Length Distribution', 'Answer Length', 'SQuAD_Validation_Answer_Lengths.png', 'dodgerblue', 15)


In [ ]:
# Plot question length distribution
question_lengths = get_length_distribution(dataset_aqa_combined['train'], 'question')
plot_histogram(question_lengths, 'Question Length Distribution', 'Question Length', 'AQA_Train_Question_Lengths.png', 'firebrick', 30)

# Plot answer length distribution
answer_lengths = get_length_distribution(dataset_aqa_combined['train'], 'answers')  # Assuming 'answer' is the key for answers
plot_histogram(answer_lengths, 'Answer Length Distribution', 'Answer Length', 'AQA_Train_Answer_Lengths.png', 'firebrick', 15)


In [ ]:
# Plot question length distribution
question_lengths = get_length_distribution(dataset_aqa_combined['validation'], 'question')
plot_histogram(question_lengths, 'Question Length Distribution', 'Question Length', 'AQA_Validation_Question_Lengths.png', 'firebrick', 30)

# Plot answer length distribution
answer_lengths = get_length_distribution(dataset_aqa_combined['validation'], 'answers')  # Assuming 'answer' is the key for answers
plot_histogram(answer_lengths, 'Answer Length Distribution', 'Answer Length', 'AQA_Vlidation_Answer_Lengths.png', 'firebrick', 15)


In [ ]:
# Plot question length distribution
question_lengths = get_length_distribution(datasets_squad_adversarial['validation'], 'question')
plot_histogram(question_lengths, 'Question Length Distribution', 'Question Length', 'SQuAD_Adversarial_Vlaidation_Question_Lengths.png', 'darkgreen', 30)

# Plot answer length distribution
answer_lengths = get_length_distribution(datasets_squad_adversarial['validation'], 'answers')  # Assuming 'answer' is the key for answers
plot_histogram(answer_lengths, 'Answer Length Distribution', 'Answer Length', 'SQuAD_Adversarial_Vlaidation_Answer_Lengths.png', 'darkgreen', 15)


In [ ]:
# Plot question length distribution
question_lengths = get_length_distribution(AdQuest, 'question')
plot_histogram(question_lengths, 'Question Length Distribution', 'Question Length', 'AdQuest_Train_Question_Lengths.png', 'goldenrod', 30)

# Plot answer length distribution
answer_lengths = get_length_distribution(AdQuest, 'answers')  # Assuming 'answer' is the key for answers
plot_histogram(answer_lengths, 'Answer Length Distribution', 'Answer Length', 'AdQuest_Train_Answer_Lengths.png', 'goldenrod', 15)


In [ ]:
import json

def remove_metadata_from_json(file_path):
    """
    Removes the 'metadata' dictionary from each JSON object in a file where each line is a separate JSON object.
    Saves the modified content to a new file.

    :param file_path: Path to the JSON file.
    """
    modified_data = []

    # Read the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            if 'metadata' in entry:
                del entry['metadata']
            modified_data.append(entry)

    # Save the modified data to a new file
    modified_file_path = file_path.replace('.json', '_modified.json')
    with open(modified_file_path, 'w') as file:
        for entry in modified_data:
            json.dump(entry, file)
            file.write('\n')  # Write each JSON object on a new line

    return modified_file_path

In [ ]:
!python3 run.py --do_train --task qa --dataset squad_v2_train.json --output_dir ./trained_model/ --model mrm8488/electra-small-finetuned-squadv2

In the next step, the model will be evaluated using the SQuAD 2.0 validation dataset. Since the model is already fine-tuned on SQuAD data, this should generate similar EM an F1 benchmarks.  

In [ ]:
from datasets import load_dataset
dataset_synQA = load_dataset("mbartolo/synQA")

# Save the 'train' split to JSON
dataset_synQA["train"].to_json('dataset_synQA_train.json')
dataset_synQA.shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import json
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
pip install jsonlines

In [ ]:
import json
import jsonlines

# Load data from the passages .jsonl file
with jsonlines.open('passages.jsonl') as reader:
    passages = {item['id']: item for item in reader}

# Load data from the question-answer .jsonl file and filter questions starting with "How many"
filtered_data = []
with jsonlines.open('generated_training_data.jsonl') as reader:
    for item in reader:
        if item['question'].lower().startswith("where"):
            passage_id = item['passage_id']
            if passage_id in passages:
                new_entry = {
                    "id": item['question_id'],
                    "title": passage_id,
                    "context": passages[passage_id]["context"],
                    "question": item['question'],
                    "answers": {
                        "text": [item["answer"]],
                        "answer_start": [item["metadata"]["answer_start"]]
                    }
                }
                filtered_data.append(new_entry)

# Write filtered data to a new JSON file
with open('filtered_questions_where.json', 'w') as outfile:
    json.dump(filtered_data, outfile, indent=4)
